# Overview

You will try to run alignment of NGS data:
- BLAST vs BWA
- Inspect and work with BAM-files
- Single end alignment of P.aeruginosa Illumina reads
- Paired end alignment of P.aeruginosa Illumina reads
- Align Illumina paired end reads ( we will do this aligment on Monday)


Find the SAM flags explained __[link text]( https://broadinstitute.github.io/picard/explain-flags.html)__

## Blast vs BWA

Let's try to align 25 Illumina reads using blast and bwa to the human genome. The "time" command infront of the commands will time how much time was actually spent on the job. Note down the time for each command, it is the third number that is written, something like m:s.ms. Firstly, move to your working directory, create a directory for this exercise and make symbolic links to the data to there. 

The data you need is the sample* FASTA and FASTQ files from directory : /exercises/alignment/blast/

Next, you run BLAST from command line:


In [ ]:
#Make a new directory for the exercises of Day2 and change your directory to it
! mkdir -p exercises/alignment/Day2
%cd exercises/alignment/Day2
! pwd

In [ ]:
! time blastn -db /exercises/alignment/blast/hg38bundle/Homo_sapiens_assembly38.fasta \
              -query /exercises/alignment/blast/sample.fasta \
              -out sample.m8 -outfmt 6 

Let's do the same for bwa. In order to run bwa mem, the format of the command is like this: 
``bwa mem SUBJECT READS1 [READS2]``. 

The "_subject_" you want to use is */home/datafiles/ngs/alignment/blast/hg38bundle/Homo_sapiens_assembly38.fasta*. You can omit the second read pair in the command here since we do not used paired-end reads. Run bwa mem for sample.fastq and time the process. Remember that BWA creates a SAM file to standard out, you probably want to redirect that into a file that ends in .sam.

**Q1. Which one was the fastest, BLAST or BWA?**

This time, we had already created the Burrows-Wheeler transform of the subject that BWA needs. If you want to create it yourself, the command is simply: 

In [ ]:
# Build index. This step is done only once. DO NOT RUN this step during the exercises. We ran the command for you.
#%%bash 
#bwa index /exercises/alignment/blast/hg38bundle/Homo_sapiens_assembly38.fasta

In [ ]:
! time bwa  mem /exercises/alignment/blast/hg38bundle/Homo_sapiens_assembly38.fasta /exercises/alignment/blast/sample.fastq > sampleReads_alignment.sam 

**SAM and BAM-files**

Now, we want to take a look at the resulting SAM file created by BWA. The SAM-format consists of two sections, header and alignments. Since SAM files are human-readable, you may look at it by simply using less (use the -S flag for added convenience).

Header lines starts with "@", "@SQ" are all the sequences you mapped against (in your reference fasta file). For the alignment the fields are: Read name, flag, reference it mapped to, position, mapping quality, cigar, mate reference map, mate position, template length, read sequence and read qualities. See SAM-specification for a thorough description.


In [ ]:
! less -S ./sampleReads_alignment.sam


Header lines starts with "@", "@SQ" are all the sequences you mapped against (in your reference fasta file). For the alignment the fields are: Read name, flag, reference it mapped to, position, mapping quality, cigar, mate reference map, mate position, template length, read sequence and read qualities. See SAM-specification for a thorough description. 

**Q2. a) At which position is the first read (i.e. the one listed at the top of the file) mapped to chromosome 21? <br> b) To which strand is it mapped?**
Hint: Check out explain flags.<br>

**Q3. a) What is the mapping quality of the first read (see column 5 of SAM file) ? <br>
    b) What do you think such a score means?**

The SAM format specification provides several information about the results of the alignment, for example *AS:i:* means the alignment score for the read or the *NM:i:3* means that the number of mismatches of the alignment is 3, which means that if we change the 3 bases in the read we can achieve a perfect match to the reference sequence (not taking indels into account).

As we always say, storing files is expensive, so you should get used to using BAM rather than SAM files. To convert SAM to BAM, we use samtools: 

In [ ]:
! samtools view -Sbh ./sampleReads_alignment.sam > sampleReads_alignment.bam 

The SAM format specification provides several information about the results of the alignment, for example *AS:i:* means the alignment score for the read or the *NM:i:3* means that the number of mismatches of the alignment is 3, which means that if we change the 3 bases in the read we can achieve a perfect match to the reference sequence (not taking indels into account).

As we always say, storing files is expensive, so you should get used to using BAM rather than SAM files. To convert SAM to BAM, we use samtools: 




In [ ]:
! samtools view -Sbh ./sampleReads_alignment.sam > sampleReads_alignment.bam 

On the command above the **-b** flag tells samtools to output BAM rather than SAM, and the **-h** flag tells it to include the headers. The **-S** flag tells samtools the input is SAM format. We could also have figured out how many reads are unmapped using the filtering option in samtools view. 

The filtering works like this: If you use *-f* FLAG, then samtools will only keep alignments with ALL the bits in FLAG set. If you use *-F* FLAG, then samtools will only keep the alignments with NONE of the bits in FLAG set. So for example, in the following command: <br>

samtools view IN.BAM -f 67 -F 12, the -f flag is 67, which is 1 + 2 + 64, which corresponds to the flags for paired reads, properly paired reads (i.e. both mates mapping to sample template), and the first read in the pair. Whereas the -F flag, 12, is 4 + 8, corresponding to unmapped reads and reads with the mate unmapped. 

So -f 67 -F 12 means reads that are paired, properly paired, and first in pair, but not unmapped, nor with their mate unmapped. 

**Q4. How many reads are unmapped? Hint: You can use samtools view with a flag to answer this question**

**Q5. How many aligned (i.e. mapped) reads do we have after filtering with these flags?**



In [ ]:

! samtools view -f 4 sampleReads_alignment.bam| wc -l
! samtools view -F 4 sampleReads_alignment.bam| wc -l

Another option is filter on the mapping quality, eg. getting all reads with a mapping quality larger than eg 30, will remove the unmapped/multi-mapped reads (mapping quality 0) and the read mapped with mapping quality 10. 

``samtools view -q 30 sample.bam | less -S ``

Now try using samtools to create a BAM file sample.mapped.bam containing headers, and only mapped reads (do not filter for quality before answering Q5!). 

In [ ]:
! samtools view -h -F 4 -q 30 sampleReads_alignment.bam > sampleReads_alignment_mapped.bam
! samtools view  sampleReads_alignment_mapped.bam| wc -l
! samtools view  sampleReads_alignment_mapped.bam| wc -l

## Paired end mapping

Let's try to add the pairs so that we have 2x25 reads and map these. Paired end reads are mapped like single end, except you pass in both the forward and the reverse FASTQ files as input: 
``bwa mem REFERENCE FORWARD_READS REVERSE_READS | samtools view -Sbh - > out.bam ``

Also, as seen in the example above, since we (almost) always prefer BAM files to SAM files, you should pipe the output of bwa mem into samtools in order to create a BAM file directly. Try it out, with the sample.fastq and sample_2.fastq files. We already have an index (BWT) of the human genome at _/exercises/alignment/blast/hg38bundle/Homo_sapiens_assembly38.fasta_

Now your sample.bam file should contain the alignment of both pairs. Have a look. The pairs are grouped together line-by-line. You can also see the flags have changed, now it also contains information on pairing, whether the paired read was mapped etc. If you want you can look up af few on explain-flags. Three extra columns are now filled, where the "=" means that the pair mapped to the same chromosome (proper pair), then the mapping position of the pair and the distance between the pairs (the size of the entire DNA fragment). 

Let's try to get reads where both paired reads map. You can look on the table (link is provided above) to see that flags - read unmapped is 4, mate unmapped is 8. So what is the flag for both read unmapped and mate unmapped? Try to filter for reads that are
properly paired
not unmapped
not with their mate unmapped
Put those in a BAM file (with headers, as always) called pairs_only.bam 

In [ ]:
! bwa  mem /exercises/alignment/blast/hg38bundle/Homo_sapiens_assembly38.fasta /exercises/alignment/blast/sample.fastq /exercises/alignment/blast/sample_2.fastq | \
        samtools view -Sbh - > sampleReads_paired_alignment.bam


In [ ]:
! samtools view -h -f 3 -F 12 sampleReads_paired_alignment.bam > sampleReads_paired_alignment.mapped.bam

In [ ]:
! samtools view sampleReads_paired_alignment.mapped.bam | wc -l

**Q6. What was the command to make the BAM file (which flags did you use)?**<br>
Finally let's try to recreate the original read files from the bam-file. Since SAM and BAM files contain both the read DNA and their Phred qualities, we can convert a SAM to a FASTQ file. To do this we are going to use a suite of Java-programs called Picard. First, convert your BAM to SAM using samtools. Then, convert SAM to FASTQ using: <br><code> picard SamToFastq INPUT=YOUR INPUT FILE FASTQ=sample.remade.fastq SECOND_END_FASTQ=sample_2.remade.fastq</code>

Check the difference of these vs. the original files. The diff program prints out lines that are different among the files. 

In [ ]:
%%bash
picard-tools SamToFastq INPUT=sampleReads_paired_alignment.mapped.bam   \
                          FASTQ=sample.remade.fastq                 \
                          SECOND_END_FASTQ=sample_2.remade.fastq

**Q7. Was there a difference between the new and original files?**

In [ ]:
! diff sample.remade.fastq /exercises/alignment/blast/sample.fastq
! diff sample_2.remade.fastq /Users/panman/Documents/Teaching/NGS_SDC_2020/course27626/alignment/blast/sample_2.fastq 

## P. aeruginosa single end Illumina reads

Let's try some reads from a study of Pseudomonas aeruginosa, an opportunistic pathogen that can live in the environment and may infect humans. Inside humans they may live in the lungs and form biofilms. The reads are from a strain that has infected ~40 patients over the last 30 years in Denmark and here it adapted to the human-host lung environment. We are going to compare it to the PAO1 reference genome which was isolated in 1955 in Australian from a wound (ie. probably it just came from the environment) and see which genes it has lost during the adaptation (assuming that the PAO1 genome is a relatively close ancestor). 

Remember that these reads were the ones we trimmed yesterday. We will also need the reference fasta which is on the following location /exercises/alignment/paeruginosa/Paeruginosa_PAO1.fna

Alignment of single end Illumina reads

bwa index the file Paeruginosa_PAO1.fna and map the FASTQ files against it. As always, convert it to BAM, preferably by piping directly into samtools. Try to map the following files to the FASTA reference file:
- Paeruginosa.fastq.gz       
- Paeruginosa.fastq.cut.gz    
- Paeruginosa.fastq.cut.trim.gz <br>
These represent raw reads, reads that have been cut for adapters, and the ones that have been trimmed for quality. Next, we can use the command samtools flagstat IN.BAM to calculate some statistics about the number of alignments with different flags. 

**Q8. Do we get more or less reads mapped after cutting and trimming?**

In [ ]:
# This command is to create an index of the Paeruginosa_PAO1.fna but you do not need to run the command.
#! bwa index /exercises/alignment/paeruginosa/Paeruginosa_PAO1.fna

In [ ]:
! bwa  mem /exercises/alignment/paeruginosa/Paeruginosa_PAO1.fna /exercises/alignment/paeruginosa/Paeruginosa.fastq.gz |   samtools view -Sbh  -  > Paeruginosa_alignment.bam

In [ ]:
%%bash
bwa  mem /exercises/alignment/paeruginosa/Paeruginosa_PAO1.fna /exercises/alignment/paeruginosa/Paeruginosa.fastq.cut.gz | \
    samtools view -Sbh  - >   Paeruginosa_trim_alignment.bam

In [ ]:
%%bash
bwa  mem /exercises/alignment/paeruginosa/Paeruginosa_PAO1.fna /exercises/alignment/paeruginosa/Paeruginosa.fastq.cut.trim.gz | \
            samtools view -Sbh  - >   Paeruginosa_cut_trim_alignment.bam

Let's look at the edit distances for each of the alignments, these are encoded in the BAM-file using the "NM:i: tag. Here we use a "perl-oneliner" to extract the edit distance for each read. Don't worry about the Perl command - you could write a simple Python script to calculate the same thing. 

``samtools view RAW_BAM_FILE | perl -ne 'if ($_ =~ m/NM:i:(\d+)/) {print $1, "\n"}' > Paeruginosa.nm ``<br>
``samtools view CUT_BAM_FILE | perl -ne 'if ($_ =~ m/NM:i:(\d+)/) {print $1, "\n"}' > Paeruginosa.cut.nm ``<br>
``samtools view TRIMMED_BAM_FILE | perl -ne 'if ($_ =~ m/NM:i:(\d+)/) {print $1, "\n"}' > Paeruginosa.cut.trim.nm ``

The plot below  demonstrates that when we cut and trim the reads, the resulting file has more reads of 0 edit distance. Whereas in the *raw* reads files and the *cut* reads file there are more reads with higher edit distance than the *cut&trim* reads.
This means that when we cut and trim the reads we remove read that had many mismatches. Having many mismatches is not always or necesserily *bad*, it shows divergence from the reference, but when they have bad qualities then they should be removed. They may mess up downstream calculations such as SNP calling.


In [ ]:
! samtools view  ./Paeruginosa_alignment.bam          | perl -ne 'if ($_ =~ m/NM:i:(\d+)/) {print $1, "\n"}' > Paeruginosa.nm
! samtools view  ./Paeruginosa_trim_alignment.bam     | perl -ne 'if ($_ =~ m/NM:i:(\d+)/) {print $1, "\n"}' > Paeruginosa.cut.nm
! samtools view  ./Paeruginosa_cut_trim_alignment.bam | perl -ne 'if ($_ =~ m/NM:i:(\d+)/) {print $1, "\n"}' > Paeruginosa.cut.trim.nm

In [ ]:
# Run an R script to generate an image with the edit distances
! Rscript --vanilla /exercises/alignment/scripts/Rplot_editDis.R ./Paeruginosa.nm  ./Paeruginosa.cut.nm ./Paeruginosa.cut.trim.nm


In [ ]:
# Visualize the image
from IPython.display import Image
Image(filename='trimming.effect.png') 